In [1]:
# section 22, beam decode from RNNG discriminative model
rnng_gold_file = '../rnng/corpora/22.auto.clean'
rnng_samples_file = 'dyer_beam/dev_pos_embeddings_beam=100.ptb_samples'
rnng_lstm_score_file = 'dyer_beam/dev_pos_embeddings_beam=100.ptb_samples.likelihoods'
rnng_gen_score_file = 'dyer_beam/dev_pos_embeddings_beam=100.samples.likelihoods2'

In [2]:
# section 22, files in 
lstm_gold_file = 'wsj/dev_22.txt.stripped'
lstm_decode_file = 'expts/beam_size=50-at_word=5.decode_all'
lstm_stderr_file = 'expts/beam_size=50-at_word=5.stderr_all'

In [3]:
import rnng_interpolate
import rnng_output_to_nbest
import rnng_threeway_interpolate
import decode_analysis

In [4]:
from ptb_reader import get_tags_tokens_lowercase

In [5]:
# RNNG order

In [29]:
with open(rnng_gold_file) as f:
    rnng_gold_tokens = [get_tags_tokens_lowercase(line)[1] for line in f]
with open(rnng_gold_file) as f:
    rnng_gold_trees = [line.strip() for line in f]

In [7]:
with open(rnng_samples_file) as f:
    rnng_indices_discrim_scores_and_parses = list(rnng_output_to_nbest.parse_rnng_file(f))

In [8]:
sample_lens = [len(l) for l in rnng_indices_discrim_scores_and_parses]

In [9]:
with open(rnng_lstm_score_file) as f:
    rnng_lstm_scores = list(rnng_interpolate.parse_likelihood_file(f))

In [10]:
with open(rnng_gen_score_file) as f:
    rnng_gen_scores = list(rnng_threeway_interpolate.parse_rnng_gen_score_file(f, sample_lens))

In [11]:
# LSTM order

In [12]:
with open(lstm_gold_file) as f:
    lstm_gold_tokens = [get_tags_tokens_lowercase(line)[1] for line in f]
with open(lstm_gold_file) as f:
    lstm_gold_trees = [line.strip() for line in f]

In [13]:
lstm_decode_instances = decode_analysis.parse_decode_output_files(lstm_gold_trees, lstm_decode_file, lstm_stderr_file)

In [14]:
rs = [tuple(s) for s in rnng_gold_tokens]
len(set(rs))

1692

In [15]:
ls = [tuple(s) for s in lstm_gold_tokens]
len(set(ls))

1692

In [16]:
import collections

In [17]:
dups = collections.defaultdict(list)
for i, e in enumerate(ls):
    dups[e].append(i)
for k, v in sorted(dups.iteritems()):
    if len(v) >= 2:
        print '%s: %r' % (k, v)

('Among', 'Big', 'Board', 'specialists', ',', 'the', 'cry', 'was', '``', 'Pull', 'your', 'offers', "''", '--', 'meaning', 'that', 'specialists', 'soon', 'expected', 'to', 'get', 'higher', 'prices', 'for', 'their', 'shares', '.'): [127, 1056]
('Many', 'market', 'makers', 'in', 'the', 'S&P', '100', 'index', 'options', 'contract', 'had', 'bullish', 'positions', 'Friday', ',', 'and', 'when', 'the', 'shutdown', 'came', 'they', 'were', 'frozen', 'with', 'huge', 'losses', '.'): [131, 1065]
('Over', 'the', 'weekend', ',', 'clearing', 'firms', 'told', 'the', 'Chicago', 'market', 'makers', 'to', 'get', 'out', 'of', 'their', 'positions', 'at', 'any', 'cost', 'Monday', 'morning', '.'): [132, 1066]
('Source', ':', 'Telerate', 'Systems', 'Inc', '.'): [1180, 1182]
('The', 'carnage', 'on', 'the', 'Chicago', 'Board', 'Options', 'Exchange', ',', 'the', 'nation', "'s", 'major', 'options', 'market', ',', 'was', 'heavy', 'after', 'the', 'trading', 'in', 'S&P', '100', 'stock-index', 'options', 'was', 'halte

In [18]:
lstm_decode_instances[127].gold_linearized

"<eos> (S (PP among (NP big board specialists )NP )PP , (NP the <unk> )NP (VP was `` (S (VP pull (NP your offers )NP )VP )S '' (PRN -- (S (VP meaning (SBAR that (S (NP specialists )NP (VP (ADVP soon )ADVP expected (S (VP to (VP get (NP higher prices )NP (PP for (NP their shares )NP )PP )VP )VP )S )VP )S )SBAR )VP )S )PRN )VP . )S <eos>"

In [19]:
lstm_decode_instances[1056].gold_linearized

"<eos> (S (PP among (NP big board specialists )NP )PP , (NP the <unk> )NP (VP was (S `` (VP pull (NP your offers )NP )VP '' )S -- (S (VP meaning (SBAR that (S (NP specialists )NP (VP (ADVP soon )ADVP expected (S (VP to (VP get (NP higher prices )NP (PP for (NP their shares )NP )PP )VP )VP )S )VP )S )SBAR )VP )S )VP . )S <eos>"

In [20]:
rnng_indices_to_lstm_indices = [
    lstm_gold_tokens.index(rnng_gt) for rnng_gt in rnng_gold_tokens
]

In [21]:
len(rnng_indices_to_lstm_indices)

1700

In [22]:
len(set(rnng_indices_to_lstm_indices))

1692

In [23]:
len(set(tuple(s) for s in lstm_gold_tokens) - set())

1692

In [24]:
# get max scoring from rnng samples under lstm probability

In [34]:
best_from_proposal = [
    max((lstm_score, parse) for (_, _, parse), lstm_score in zip(ipp, scores))
    for ipp, scores in zip(rnng_indices_discrim_scores_and_parses, rnng_lstm_scores)
]
# for i, (ipp, lstm_scores) in enumerate(zip(rnng_indices_discrim_scores_and_parses, rnng_lstm_scores)):
        

In [35]:
best_from_proposal[0]

(-147.777460351,
 "(S (NP (NP (JJ Influential) (NNS members)) (PP (IN of) (NP (DT the) (NNP House) (NNP Ways) (CC and) (NNP Means) (NNP Committee)))) (VP (VBD introduced) (NP (NP (NN legislation)) (SBAR (WHNP (WDT that)) (S (VP (MD would) (VP (VB restrict) (SBAR (WHADVP (WRB how)) (S (NP (DT the) (JJ new) (JJ savings-and-loan) (NN bailout) (NN agency)) (VP (MD can) (VP (VB raise) (NP (NN capital)) (, ,) (S (VP (VBG creating) (NP (NP (DT another) (JJ potential) (NN obstacle)) (PP (TO to) (NP (NP (NP (DT the) (NN government) (POS 's)) (NN sale)) (PP (IN of) (NP (JJ sick) (NNS thrifts)))))))))))))))))) (. .))")

In [36]:
lstm_decode_instances[rnng_indices_to_lstm_indices[0]]

DecodeInstance(index=764, gold_linearized="<eos> (S (NP (NP influential members )NP (PP of (NP the house ways and means committee )NP )PP )NP (VP introduced (NP (NP legislation )NP (SBAR (WHNP that )WHNP (S (VP would (VP restrict (SBAR (WHADVP how )WHADVP (S (NP the new savings-and-loan bailout agency )NP (VP can (VP raise (NP capital )NP )VP )VP )S )SBAR , (S (VP creating (NP (NP another potential obstacle )NP (PP to (NP (NP (NP the government 's )NP sale )NP (PP of (NP sick thrifts )NP )PP )NP )PP )NP )VP )S )VP )VP )S )SBAR )NP )VP . )S <eos>", pred_linearized="<eos> (S (S (S (S (NP (NP (ADJP (ADJP influential )ADJP (SBAR (S (NP (NP members )NP (PP of (NP the house ways and means committee )NP )PP )NP (VP introduced (NP (NP legislation )NP (SBAR (WHNP that )WHNP (S (VP would (VP restrict (SBAR (WHADVP how )WHADVP (S (NP the new savings-and-loan bailout agency )NP (VP can (VP raise (NP capital )NP )VP )VP )S )SBAR )VP )VP )S , (S (VP creating (NP (NP another potential obstacle )NP (P

In [37]:
best_proposal_fname = '/tmp/best_from_proposal.out'
best_proposal_gold_fname = '/tmp/best_from_proposal_and_gold.out'
best_proposal_decode_fname = '/tmp/best_from_proposal_and_decode.out'
best_proposal_gold_decode_fname = '/tmp/best_from_proposal_gold_and_decode.out'

In [50]:
only_decode_fname = '/tmp/only_decode.out'

In [39]:
di = lstm_decode_instances[0]

In [40]:
di.gold_ptb

'(S1 (S (NP (NNP Chicago) (NN investor) (NNP William) (NNP Farley)) (VP (VBD agreed) (S (VP (TO to) (VP (VB sell) (NP (NP (CD three) (NNS divisions)) (PP (IN of) (NP (NNP Cluett) (NNP Peabody) (CC &) (NNP Co.)))) (PP (IN for) (NP (QP (RB about) ($ $) (CD 600) (CD million)))) (PP (TO to) (NP (NP (NNP Bidermann) (NNP S.A.)) (, ,) (NP (NP (DT a) (ADJP (RB closely) (VBN held)) (NN clothing) (NN maker)) (VP (VBN based) (PP (IN in) (NP (NNP Paris))))))))))) (. .)))'

In [46]:
N_sents = len(rnng_indices_discrim_scores_and_parses)

In [49]:
import evaluate

In [48]:
def percent_tuple_str(n):
    return "%s / %s (%0.2f%%)" % (n, N_sents, float(n) * 100 / N_sents)

In [56]:
decode_beats_proposal = 0
decode_beats_gold = 0
decode_beats_gold_and_proposal = 0
# just the samples
with open(best_proposal_fname, 'w') as f_proposal,\
    open(best_proposal_gold_fname, 'w') as f_proposal_gold,\
    open(best_proposal_decode_fname, 'w') as f_proposal_decode,\
    open(best_proposal_gold_decode_fname, 'w') as f_proposal_gold_decode,\
    open(only_decode_fname, 'w') as f_decode:
    for i, (best_proposal_score, best_proposal) in enumerate(best_from_proposal):
        gold_parse = rnng_gold_trees[i]
        decode_instance = lstm_decode_instances[rnng_indices_to_lstm_indices[i]]
        assert(get_tags_tokens_lowercase(decode_instance.gold_ptb)[1] == get_tags_tokens_lowercase(gold_parse)[1])
#         print(i, best_proposal_score, decode_instance.pred_score, decode_instance.gold_score)
        f_proposal.write("%s\n" % best_proposal)
        
        f_decode.write("%s\n" % decode_instance.gold_ptb)
        
        if decode_instance.pred_score > best_proposal_score + 1e-3:
            decode_beats_proposal += 1
            
        if decode_instance.pred_score > decode_instance.gold_score + 1e-3:
            decode_beats_gold += 1
            
        if decode_instance.pred_score > decode_instance.gold_score + 1e-3 and decode_instance.pred_score > best_proposal_score + 1e-3:
            decode_beats_gold_and_proposal += 1
        
        f_proposal_decode.write("%s\n" % max([
                    (best_proposal_score, best_proposal),
                    (decode_instance.pred_score, decode_instance.pred_ptb)
                ], key=lambda t: t[0])[1])
        
        f_proposal_gold.write("%s\n" % max([
                    (best_proposal_score, best_proposal),
                    (decode_instance.gold_score, decode_instance.gold_ptb)
                ], key=lambda t: t[0])[1])
        
        f_proposal_gold_decode.write("%s\n" % max([
                    (best_proposal_score, best_proposal),
                    (decode_instance.pred_score, decode_instance.pred_ptb),
                    (decode_instance.gold_score, decode_instance.gold_ptb)
                ], key=lambda t: t[0])[1])
            
print("decode beats proposal:\t" + percent_tuple_str(decode_beats_proposal))
print("decode beats gold:\t" + percent_tuple_str(decode_beats_gold))
print("decode beats gold and prop:\t" + percent_tuple_str(decode_beats_gold_and_proposal))
print
print("gold sanity check (R, P, F1, exact match):")
print(evaluate.eval_b(rnng_gold_file, only_decode_fname))
print("rescore proposal (R, P, F1, exact match):")
print(evaluate.eval_b(rnng_gold_file, best_proposal_fname))
print("rescore proposal+gold (R, P, F1, exact match):")
print(evaluate.eval_b(rnng_gold_file, best_proposal_gold_fname))
print("rescore proposal+decode (R, P, F1, exact match):")
print(evaluate.eval_b(rnng_gold_file, best_proposal_decode_fname))
print("rescore proposal+decode+gold (R, P, F1, exact match):")
print(evaluate.eval_b(rnng_gold_file, best_proposal_gold_decode_fname))

decode beats proposal:	251 / 1700 (14.76%)
decode beats gold:	360 / 1700 (21.18%)
decode beats gold and prop:	86 / 1700 (5.06%)

gold sanity check (R, P, F1, exact match):
(100.0, 99.99, 100.0, 99.88)
rescore proposal (R, P, F1, exact match):
(93.49, 93.6, 93.54, 49.26)
rescore proposal+gold (R, P, F1, exact match):
(94.12, 94.18, 94.15, 53.27)
rescore proposal+decode (R, P, F1, exact match):
(93.52, 93.56, 93.54, 49.47)
rescore proposal+decode+gold (R, P, F1, exact match):
(94.05, 94.06, 94.06, 52.59)
